***
## Create a multi fasta file : 

In [4]:
import os
import pandas as pd
from Bio import SeqIO

path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
path_annotation = f"{path_project}/annotation_classic"

missing_dpos = pd.read_csv(f"{path_project}/full_missed_dpo.77_phages.tsv", sep = " ", header = 0)


In [11]:
with open(f"{path_project}/full_missing_dpos.multi.fasta", "w") as outfile :
    for _, row in missing_dpos.iterrows() : 
        path_fasta = f"{path_project}/annotation_classic/{row['Phage']}/{row['Phage']}.multi.annotated.faa"
        fasta_file = SeqIO.parse(path_fasta, "fasta")
        header = f"{row['Phage']} {row['CDS']},"
        for record in fasta_file : 
            if record.description.count(header) > 0 : 
                outfile.write(f">{record.description}\n{record.seq}\n")
            

    

> Move it to the server :

In [ ]:
rsync -avzhe ssh \
/media/concha-eloko/Linux/77_strains_phage_project/full_missing_dpos.multi.fasta \
conchae@garnatxa.srv.cpd:/home/conchae/PhageDepo_pdb

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/PhageDepo_pdb/full_missing_dpos_esmfold \
/media/concha-eloko/Linux/77_strains_phage_project/


***
# Create the multi fasta : 


In [5]:
import os 

boundaries_df = pd.read_csv(f"{path_project}/full_missed_dpo.77_phages.boundaries.tsv", sep = "\t", header =0)
boundaries_df.drop_duplicates(subset = ["Phage CDS"], inplace = True)

In [6]:
def get_domain(seq , boundaries_info) :
    if str(boundaries_info).count(";") == 0 :
        start , end = str(boundaries_info).split("-")[0] , str(boundaries_info).split("-")[1]
        domain_seq = seq[int(start):int(end)]
        return domain_seq
    else :
        full_seq = ""
        for _,segment_info in enumerate(str(boundaries_info).split(";")) : 
            start , end = segment_info.split("-")[0] , segment_info.split("-")[1]
            domain_seq = seq[int(start): int(end)]
            full_seq = full_seq + domain_seq
        return domain_seq
    

In [7]:
# Write the multi fasta file : 
with open(f"{path_project}/missed_dpo.77_phages.domains.multi.fasta" , "w") as outfile :
    for _,row in boundaries_df.iterrows() : 
        phage , cds = row["Phage CDS"].split(" ")[0] , row["Phage CDS"].split(" ")[1]
        boundaries = row["Boundaries"]
        if boundaries != "NaN" : 
            path_fasta = f"{path_annotation}/{phage}/{phage}.multi.annotated.faa"
            fasta_file = SeqIO.parse(path_fasta , "fasta")
            for index, record in enumerate(fasta_file) : 
                if record.description.count(row["Phage CDS"]) > 0 :
                    seq = record.seq
                    domain_seq = get_domain(seq , boundaries)
                    outfile.write(f">{phage}_{cds}__{boundaries}\n{domain_seq}\n")
                    print(f"{phage} {cds}\t{phage}{index}")

            
    

K10PH82C1 cds_45	K10PH82C144
K11PH164C1 cds_39	K11PH164C138
K12P1_1 cds_43	K12P1_142
K13PH07C1L cds_54	K13PH07C1L53
K13PH07C1S cds_53	K13PH07C1S52
K15PH90 cds_49	K15PH9048
K16PH164C3 cds_43	K16PH164C342
K19PH14C4P1 cds_43	K19PH14C4P142
K1PH164C1 cds_53	K1PH164C152
K2069PH1 cds_25	K2069PH124
K22PH164C1 cds_50	K22PH164C149
K24PH164C1 cds_55	K24PH164C154
K25PH129C1 cds_56	K25PH129C155
K26PH128C1 cds_44	K26PH128C143
K27PH129C1 cds_43	K27PH129C142
K35PH164C3 cds_43	K35PH164C342
K37PH164C1 cds_41	K37PH164C140
K39PH122C2 cds_50	K39PH122C249
K40PH129C1 cds_52	K40PH129C151
K42PH8 cds_43	K42PH842
K42PH8 cds_48	K42PH847
K43PH164C1 cds_35	K43PH164C134
K44PH129C1 cds_45	K44PH129C144
K48PH164C1 cds_43	K48PH164C142
K51PH129C1 cds_56	K51PH129C155
K56PH164C1 cds_43	K56PH164C142
K58PH129C2 cds_40	K58PH129C239
K59PH2 cds_46	K59PH245
K61PH164C1 cds_52	K61PH164C151
K66PH128C1 cds_55	K66PH128C154
K71PH129C1 cds_51	K71PH129C150
K72PH164C2 cds_51	K72PH164C250
K74PH129C2 cds_46	K74PH129C245
K80PH1317a cds_47	K

> Move it to the server :

In [ ]:
rsync -avzhe ssh \
/media/concha-eloko/Linux/77_strains_phage_project/missed_dpo.77_phages.domains.multi.fasta \
conchae@garnatxa.srv.cpd:/home/conchae/77_strains_phage_project


rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/77_strains_phage_project/missed_dpo.77_phages.domains.multi.fasta.esm2_out \
/media/concha-eloko/Linux/77_strains_phage_project


In [8]:
import torch
import os 
import pandas as pd
from tqdm import tqdm 


path_esm = "/media/concha-eloko/Linux/77_strains_phage_project/missed_dpo.77_phages.domains.multi.fasta.esm2_out"

embeddings_esm = {}
for file in tqdm(os.listdir(path_esm)) :
    path_file = f"{path_esm}/{file}"
    index = file.split(".")[0]
    embb = torch.load(f"{path_file}")["mean_representations"][33].tolist()
    embeddings_esm[index] = embb
    
with open(f"/media/concha-eloko/Linux/77_strains_phage_project/missed_dpo.77_phages.domains.esm2.embedding.csv" , "w") as outfile :
    for index in tqdm(embeddings_esm) :
        outfile.write(f"{index},")
        for _,  emb in enumerate(embeddings_esm[index]) :
            outfile.write(f"{emb},")
        outfile.write("\n")

100%|██████████████████████████████████████████| 49/49 [00:00<00:00, 935.05it/s]
